# <font color=4CBB17>**1. Sistemas basados en popularidad**</font>

In [1]:
# CONECTAR CON DRIVE
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import sys
path ='/content/drive/MyDrive/cod/LEA3_Marketing'
os.chdir(path) ## volver la carpeta de repositorio directorio de trabajo
sys.path.append(path) ## agregarla al path, para leer archivos propios como paquetes

In [1]:
#!pip install ipywidgets
import numpy as np
import pandas as pd
import sqlite3 as sql
from sklearn.preprocessing import MinMaxScaler
from ipywidgets import interact ## para análisis interactivo
from sklearn import neighbors ### basado en contenido un solo producto consumido
import joblib

# CREAR CONEXIÓN CON LA BASE DE DATOS db_movies
con = sql.connect('data/db_movies')

# CREAR EL CURSOR
cur = con.cursor() ## se crea el cursor, que es el otro tipo de conexión para ejecutar las consultas

In [2]:
# VERIFICAR LOS NOMBRES DE TODAS LAS TABLAS QUE HAY EN LA BASE DE DATOS
cur.execute(""" select name from sqlite_master where type= 'table'  """)
cur.fetchall()

[('ratings',),
 ('movies',),
 ('usuarios_selectos',),
 ('Pelis_selectas',),
 ('ratings_final',),
 ('movies_final',),
 ('full_ratings',)]

In [15]:
full_ratings = pd.read_sql('SELECT * FROM full_ratings', con)
full_ratings.head(5)

,movie_id,movie_title,user_id,movie_rating,movie_genres
0,1,Toy Story (1995),1,4.0,Adventure|Animation|Children|Comedy|Fantasy
1,3,Grumpier Old Men (1995),1,4.0,Comedy|Romance
2,6,Heat (1995),1,4.0,Action|Crime|Thriller
3,47,Seven (a.k.a. Se7en) (1995),1,5.0,Mystery|Thriller
4,50,"Usual Suspects, The (1995)",1,5.0,Crime|Mystery|Thriller


In [17]:
full_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36767 entries, 0 to 36766
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   movie_id      36767 non-null  int64  
 1   movie_title   36767 non-null  object 
 2   user_id       36767 non-null  int64  
 3   movie_rating  36767 non-null  float64
 4   movie_genres  36767 non-null  object 
dtypes: float64(1), int64(2), object(2)
memory usage: 1.4+ MB


In [52]:
# Primero es necesario separar el título y el año de la tabla "movies" en columnas diferentes

# Se identifico que hay espacios en blanco al final


In [18]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from mlxtend.preprocessing import TransactionEncoder

# Paso 0: df_delete ya obtenido previamente con SQL

# Función 1: separar géneros y convertir a binario con TransactionEncoder
def split_and_encode_genres(df):
    genres = df['movie_genres'].str.split('|')
    te = TransactionEncoder()
    genres_bin = te.fit_transform(genres)
    genres_df = pd.DataFrame(genres_bin, columns=te.columns_)

    # Eliminar "(no genres listed)" si existe
    if '(no genres listed)' in genres_df.columns:
        valid_rows = ~genres_df['(no genres listed)'] # La virgulilla me convierte lo TRUE en FALSE y viceversa
        df = df.loc[valid_rows].reset_index(drop=True) # Filtro por las columnas que si tienen genero
        genres_df = genres_df.loc[valid_rows].drop(columns='(no genres listed)').reset_index(drop=True)

    # Eliminar columna original 'genres' y unir los géneros codificados
    df = df.drop(columns='movie_genres').reset_index(drop=True) # Elimina la columna original 'genres' del df
    return pd.concat([df, genres_df], axis=1)

# Función 2: extraer título y año
def extract_title_and_year(df):
    year = df['movie_title'].str.extract(r'\((\d{4})\)$')
    year.columns = ['movie_year']
    title = df['movie_title'].str.replace(r'\s*\(\d{4}\)$', '', regex=True)
    title.name = 'movie_title'
    df = df.drop(columns='movie_title')
    df = pd.concat([df.reset_index(drop=True), title.reset_index(drop=True), year.reset_index(drop=True)], axis=1)
    return df

# Función 4: eliminar registros con year == NaN
def remove_nan_years(df):
    return df[df['movie_year'].notna()].reset_index(drop=True)

# Construcción del pipeline
pipeline = Pipeline(steps=[
    ('genres_transform', FunctionTransformer(split_and_encode_genres, validate=False)),
    ('extract_title_year', FunctionTransformer(extract_title_and_year, validate=False)),
    ('remove_nan_years', FunctionTransformer(remove_nan_years, validate=False))
])

# Aplicar el pipeline
db_movies_final = pipeline.fit_transform(full_ratings)

In [19]:
db_movies_final

,movie_id,user_id,movie_rating,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,movie_title,movie_year
0,1,1,4.0,False,True,True,True,True,False,False,...,False,False,False,False,False,False,False,False,Toy Story,1995
1,3,1,4.0,False,False,False,False,True,False,False,...,False,False,False,True,False,False,False,False,Grumpier Old Men,1995
2,6,1,4.0,True,False,False,False,False,True,False,...,False,False,False,False,False,True,False,False,Heat,1995
3,47,1,5.0,False,False,False,False,False,False,False,...,False,False,True,False,False,True,False,False,Seven (a.k.a. Se7en),1995
4,50,1,5.0,False,False,False,False,False,True,False,...,False,False,True,False,False,True,False,False,"Usual Suspects, The",1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36762,592,609,3.0,True,False,False,False,False,True,False,...,False,False,False,False,False,True,False,False,Batman,1989
36763,742,609,3.0,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,Thinner,1996
36764,786,609,3.0,True,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,Eraser,1996
36765,892,609,3.0,False,False,False,False,True,False,False,...,False,False,False,True,False,False,False,False,Twelfth Night,1996


In [20]:
# la tabla "full_ratings" ya existe en la base de datos SQL, se eliminará
# y se creará una nueva tabla con los datos actuales de db_movies_final
db_movies_final.to_sql("full_ratings", con, index=False, if_exists='replace')


36767

In [ ]:
#pd.set_option('display.max_columns', None)  # Muestra todas las columnas
#pd.set_option('display.width', 1000)  # Aumenta el ancho de la línea

## TOP 10 Peliculas con mejores calificaciones y mas calificadas

Se uso una fórmula, que es una adaptación del ranking ponderado de IMDb (o Bayesian average). Se usa para evitar que películas con muy pocos votos, pero notas muy altas, aparezcan en los primeros lugares.

https://stats.stackexchange.com/questions/189658/what-are-good-resources-on-bayesian-rating

Weighted rating (WR) = (v ÷ (v+m)) × R + (m ÷ (v+m)) × C , where:

* R = average for the movie (mean) = (Rating)
* v = number of votes for the movie = (votes)
* m = minimum votes required to be listed in the Top 
* C = the mean vote across the whole report 

>Se usa pesos para dar a los más calificados mejor puntuacion asi como a los mejor calificados mas puntuacion, entonces una pelicula con muchas calificaciones y bien calificadas tendra gran oportunidad de hacer parte de este top 10 de popularidad

In [22]:
# Calcular el promedio de calificación (C) y el percentil 75 de cantidad de calificaciones (m)

# Conteo de rantings que agrupa las calificaciones por movieId en donde cada fila
# representa una película y muestra cuántas veces ha sido calificada
rating_counts_df = pd.read_sql("""
    SELECT COUNT(*) AS rating_count
    FROM full_ratings
    GROUP BY movie_id
""", con)

# Calificacion promedio / rating promedio
avg_rating_df = pd.read_sql("""
    SELECT AVG(movie_rating) AS C
    FROM full_ratings
""", con)

# Extraccion de calificación promedio
C = avg_rating_df['C'].iloc[0]

# calculo del percentil 75.
m = rating_counts_df['rating_count'].quantile(0.75)

print(f"Rating promedio (C): {C}")
print(f"Percentil 75 del conteo de calificaciones (m): {m}")


Rating promedio (C): 3.731783936682351
Percentil 75 del conteo de calificaciones (m): 20.0


A continuación, se hará la busqueda en SQL y se va a crear la columna que será la calificación ponderada, ya que se considera es mejor que la simple (promedio), porque reduce el sesgo de películas con pocos votos, dando más relevancia a aquellas tienen más calificaciones. En este caso,

In [26]:
## se prepara el query donde se calcula la calificación ponderada dando uso a
# los productos con mejor calificación ponderada, es decir, los más calificados y con mayor calificación
query = f"""
    SELECT 
        movie_year,
        movie_title,
        COUNT(*) AS rating_count,
        ((COUNT(*) / (COUNT(*) + {m})) * AVG(movie_rating) + ({m} / (COUNT(*) + {m})) * {C}) AS weighted_rating
    FROM full_ratings
    WHERE movie_rating > 0
    GROUP BY movie_title
    HAVING rating_count > 30
    ORDER BY weighted_rating DESC
    LIMIT 10
"""
df = pd.read_sql(query, con)
df

,movie_year,movie_title,rating_count,weighted_rating
0,1994,"Shawshank Redemption, The",224,4.402195
1,1977,Star Wars: Episode IV - A New Hope,154,4.227791
2,1998,American History X,80,4.206357
3,1995,"Usual Suspects, The",127,4.194801
4,1980,Star Wars: Episode V - The Empire Strikes Back,127,4.194801
5,1983,Star Wars: Episode VI - Return of the Jedi,115,4.189894
6,1993,Schindler's List,148,4.182355
7,1994,Forrest Gump,223,4.175455
8,2009,Inglourious Basterds,53,4.173091
9,1987,"Princess Bride, The",81,4.169660


In [36]:
#### las mejores peliculas calificados en los ultimos 10 años registrados ###

pd.read_sql("""
    -- Crear una tabla temporal con películas rankeadas por año y calificación, debe ser con WITH porque
    -- esto es SQLite entonces no se puede usar SELECT DISTINCT        
            
    WITH ranked_movies AS ( 
    SELECT 
        movie_title,
        movie_year,
        movie_id,
        movie_rating,
        ROW_NUMBER() OVER (PARTITION BY movie_year ORDER BY movie_rating DESC) AS rank
        -- Asigna un número de fila ordenado por calificación descendente dentro de cada año    
    FROM full_ratings
)
SELECT 
    movie_title,
    movie_year,
    movie_rating AS best_rating
FROM ranked_movies
WHERE rank = 1  -- Solo tomar la mejor calificada por año
ORDER BY movie_year DESC
LIMIT 10; --las 10 películas más recientes
""", con)

,movie_title,movie_year,best_rating
0,Avengers: Infinity War - Part I,2018,5.0
1,Thor: Ragnarok,2017,5.0
2,10 Cloverfield Lane,2016,5.0
3,Spectre,2015,5.0
4,Interstellar,2014,5.0
5,Prisoners,2013,5.0
6,"Dark Knight Rises, The",2012,5.0
7,Intouchables,2011,5.0
8,Despicable Me,2010,5.0
9,Moon,2009,5.0


In [ ]:
#######################################################################
######## 2.1 Sistema de recomendación basado en contenido un solo producto - Manual ########
#######################################################################

movies_2=pd.read_sql('select * from movies_final', con )

movies_2.info()
movies_2['año_estreno']=movies_2.año_estreno.astype('int')
movies_2.info()

##### escalar para que año esté en el mismo rango ###

sc=MinMaxScaler()
movies_2[["year_sc"]]=sc.fit_transform(movies_2[['year_pub']])



## eliminar filas que no se van a utilizar ###

movies_2_dum1=movies_2.drop(columns=['isbn','i_url','year_pub','book_title']) ## ingresar las columnas a quitar ###

#### convertir a dummies

movies_2_dum1['book_author'].nunique()
movies_2_dum1['publisher'].nunique()

col_dum=['book_author','publisher']
movies_2_dum2=pd.get_dummies(movies_2_dum1,columns=col_dum)
movies_2_dum2.shape

joblib.dump(movies_2_dum2,"salidas\\movies_2_dum2.joblib") ### para utilizar en segundos modelos


